In [29]:
import pickle
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [30]:
axis_label_size = 18
axis_ticks_size = 14
dimension_label_size = axis_label_size*1
dimension_ticks_size = axis_ticks_size*1
cbar_label_size = 16
cbar_ticks_size = 14
legend_title_size = 'x-large'
legend_entry_size = 14

## Plot experiment 1
For this experiment the parameters "a", "A1", and "A2" have been optimized. The optimization used 50 individuals and 100 generations (only 5 generations reached). 

In [31]:
file_storage_path = './results/experiment_1.pkl'

with open(file_storage_path, 'rb') as f:
    data = pickle.load(f)

# seperate best individual and data
best_individual = data[1::2]
data = data[::2]
# select only data from best individual
all_data = []
data_best_individual = []
for counter, individual in enumerate(data):
    all_data = np.append(all_data, individual)
    data_best_individual.append(individual[best_individual[counter]])
    
df_plot = pd.DataFrame.from_records(all_data)

In [32]:
df_plot.sort_values(by='fitness', ascending=False, inplace=True)
df_plot.head()

,individual,a,A1,A2,fitness
150,1,-1235.386034,1.190189,0.313551,93.352221
81,32,-1235.386034,1.190189,0.313551,91.065705
100,1,-1235.386034,1.190189,0.313551,90.753777
200,1,-1235.386034,1.190189,0.313551,89.860920
84,35,-397.073337,-9.126374,5.700049,85.946085


In [33]:
min_acc = np.round(np.nanmin(df_plot["fitness"]),-1)
min_bar = (int(str(min_acc)[0])-1)*10**(len(str(int(min_acc)))-1)

max_acc = np.round(np.nanmax(df_plot["fitness"]),-1)
max_bar = (int(str(max_acc)[0])+1)*10**(len(str(int(min_acc)))-1)

In [34]:
# fix number of ticks and set precision
tickvals_a = np.linspace(np.nanmin(df_plot['a'])*1.05, np.nanmax(df_plot['a'])*1.05, 10)
tickvals_a = tickvals_a.round(0)

tickvals_A1 = np.linspace(np.nanmin(df_plot['A1'])*1.05, np.nanmax(df_plot['A1'])*1.05, 10)
tickvals_A1 = tickvals_A1.round(2)

tickvals_A2 = np.linspace(np.nanmin(df_plot['A2'])*1.05, np.nanmax(df_plot['A2'])*1.05, 10)
tickvals_A2 = tickvals_A2.round(2)

In [35]:

fig = go.Figure(layout=dict(width=1000, height=500, margin=go.layout.Margin(l=50, r=20, b=20, t=50, pad=0)),
                data=go.Parcoords(
    line=dict(color=df_plot['fitness'],
                colorscale=px.colors.diverging.Portland,
                showscale=True,
                cmin=0,
                cmax=100,
                colorbar={"title": dict(text="Accuracy (%)", side="right", font=dict(size=cbar_label_size)),
                          "tickvals": np.arange(0, 100+1, 10),
                          "tickfont": dict(size=cbar_ticks_size)}),
    dimensions=list([
        dict(tickvals=tickvals_a,
            label="a", values=df_plot['a']),
        dict(tickvals=tickvals_A1,
            label='A1', values=df_plot['A1']),
        dict(tickvals=tickvals_A2,
            label='A2', values=df_plot['A2']),
        dict(range=[0, 100],
             tickvals=np.arange(0, 100+1, 20),
             label='Accuracy (%)', values=df_plot['fitness'])
    ]),
    labelfont=dict(size=dimension_label_size),
    tickfont=dict(size=dimension_ticks_size)
)
)
image_filename = "bio_inspired_optimization_1"

fig.write_image(f"./plots/{image_filename}.pdf")
fig.write_image(f"./plots/{image_filename}.png")
fig.show()
print("Nbr. trials: {}" .format(len(df_plot)))
print("Nbr. generations: {}" .format(int(len(df_plot)/50)))


Nbr. trials: 250
Nbr. generations: 5


### Highlight top 10%

In [36]:
df_plot1 = df_plot.iloc[:int(len(df_plot)*0.1), :]
df_plot2 = df_plot.iloc[int(len(df_plot)*0.1):, :]

In [37]:
df_plot1.head()


,individual,a,A1,A2,fitness
150,1,-1235.386034,1.190189,0.313551,93.352221
81,32,-1235.386034,1.190189,0.313551,91.065705
100,1,-1235.386034,1.190189,0.313551,90.753777
200,1,-1235.386034,1.190189,0.313551,89.860920
84,35,-397.073337,-9.126374,5.700049,85.946085


In [38]:
print(min(df_plot1["fitness"]))
print(max(df_plot1["fitness"]))

80.56032509157511
93.35222069597069


In [39]:
df_plot2.head()

,individual,a,A1,A2,fitness
144,45,-513.636364,-7.272727,-1.727273,80.500229
195,46,-631.818182,-5.681818,-0.272727,80.411516
232,33,-514.555475,-38.705715,0.375445,80.256983
143,44,-293.030303,-7.500000,-0.575758,80.119620
8,9,-726.363636,-10.909091,2.878788,80.022321


In [40]:
min_top_ten = min(df_plot1["fitness"])*1E-2
max_top_ten = max(df_plot1["fitness"])*1E-2
min_all = min(df_plot["fitness"])*1E-2
max_all = max(df_plot["fitness"])*1E-2

fig = go.Figure(layout=dict(width=1000, height=500, margin=go.layout.Margin(l=50, r=20, b=20, t=50, pad=0)),
                data=go.Parcoords(
    line=dict(color=df_plot['fitness'],
                colorscale=[[0, 'white'], [0.9,'lightgrey'], [0.91,'blue'], [0.96, 'orange'], [1, 'red']],
                showscale=True,
                cmin=min_all*1E2,
                cmax=max_all*1E2,
                colorbar={"title": dict(text="Accuracy (%)", side="right", font=dict(size=cbar_label_size)),
                          "tickvals": np.arange(10, 100+1, 10),
                          "tickfont": dict(size=cbar_ticks_size)}),
    dimensions=list([
        dict(tickvals=tickvals_a,
            label="a", values=df_plot['a']),
        dict(tickvals=tickvals_A1,
            label='A1', values=df_plot['A1']),
        dict(tickvals=tickvals_A2,
            label='A2', values=df_plot['A2']),
        dict(range=[0, 100],
             tickvals=np.arange(0, 100+1, 20),
             label='Accuracy (%)', values=df_plot['fitness'])
    ]),
    labelfont=dict(size=dimension_label_size),
    tickfont=dict(size=dimension_ticks_size)
)
)
image_filename = "bio_inspired_optimization_1_selection"

fig.write_image(f"./plots/{image_filename}.pdf")
fig.write_image(f"./plots/{image_filename}.png")
fig.show()
print("Nbr. trials highlighted: {}" .format(len(df_plot1)))
print("Nbr. generations: {}" .format(int(len(df_plot)/50)))


Nbr. trials highlighted: 25
Nbr. generations: 5


## Plot experiment 2
For this experiment the parameters "a", "A1", "A2", "b", and "G" have been optimized. The optimization used 10 individuals and 100 generations.

In [41]:
file_storage_path = './results/experiment_2.pkl'

with open(file_storage_path, 'rb') as f:
    data = pickle.load(f)

# seperate best individual and data
best_individual = data[1::2]
data = data[::2]
# select only data from best individual
all_data = []
data_best_individual = []
for counter, individual in enumerate(data):
    all_data = np.append(all_data, individual)
    data_best_individual.append(individual[best_individual[counter]])
    
df_plot = pd.DataFrame.from_records(all_data)

In [42]:
df_plot.sort_values(by='fitness', ascending=False, inplace=True)

min_acc = np.round(np.nanmin(df_plot["fitness"]),-1)
min_bar = (int(str(min_acc)[0])-1)*10**(len(str(int(min_acc)))-1)

max_acc = np.round(np.nanmax(df_plot["fitness"]),-1)
max_bar = (int(str(max_acc)[0])+1)*10**(len(str(int(min_acc)))-1)

# fix number of ticks and set precision
tickvals_a = np.linspace(np.nanmin(df_plot['a'])*1.05, np.nanmax(df_plot['a'])*1.05, 10)
tickvals_a = tickvals_a.round(0)

tickvals_A1 = np.linspace(np.nanmin(df_plot['A1'])*1.05, np.nanmax(df_plot['A1'])*1.05, 10)
tickvals_A1 = tickvals_A1.round(2)

tickvals_A2 = np.linspace(np.nanmin(df_plot['A2'])*1.05, np.nanmax(df_plot['A2'])*1.05, 10)
tickvals_A2 = tickvals_A2.round(2)

tickvals_b = np.linspace(np.nanmin(df_plot['b'])*1.05, np.nanmax(df_plot['b'])*1.05, 10)
tickvals_b = tickvals_b.round(0)

tickvals_G = np.linspace(np.nanmin(df_plot['G'])*1.05, np.nanmax(df_plot['G'])*1.05, 10)
tickvals_G = tickvals_G.round(0)

In [43]:

fig = go.Figure(layout=dict(width=1000, height=500, margin=go.layout.Margin(l=50, r=20, b=20, t=50, pad=0)),
                data=go.Parcoords(
    line=dict(color=df_plot['fitness'],
                colorscale=px.colors.diverging.Portland,
                showscale=True,
                cmin=0,
                cmax=100,
                colorbar={"title": dict(text="Accuracy (%)", side="right", font=dict(size=cbar_label_size)),
                          "tickvals": np.arange(0, 100+1, 10),
                          "tickfont": dict(size=cbar_ticks_size)}),
    dimensions=list([
        dict(tickvals=tickvals_a,
            label="a", values=df_plot['a']),
        dict(tickvals=tickvals_A1,
            label='A1', values=df_plot['A1']),
        dict(tickvals=tickvals_A2,
            label='A2', values=df_plot['A2']),
        dict(tickvals=tickvals_b,
            label='b', values=df_plot['b']),
        dict(tickvals=tickvals_G,
            label='G', values=df_plot['G']),
        dict(range=[0, 100],
             tickvals=np.arange(0, 100+1, 20),
             label='Accuracy (%)', values=df_plot['fitness'])
    ]),
    labelfont=dict(size=dimension_label_size),
    tickfont=dict(size=dimension_ticks_size)
)
)
image_filename = "bio_inspired_optimization_2"

fig.write_image(f"./plots/{image_filename}.pdf")
fig.write_image(f"./plots/{image_filename}.png")
fig.show()
print("Nbr. trials: {}" .format(len(df_plot)))
print("Nbr. generations: {}" .format(int(len(df_plot)/10)))


Nbr. trials: 340
Nbr. generations: 34


### Highlight top 10%

In [44]:
df_plot1 = df_plot.iloc[:int(len(df_plot)*0.1), :]
df_plot2 = df_plot.iloc[int(len(df_plot)*0.1):, :]

In [45]:
min_top_ten = min(df_plot1["fitness"])*1E-2
max_top_ten = max(df_plot1["fitness"])*1E-2
min_all = min(df_plot["fitness"])*1E-2
max_all = max(df_plot["fitness"])*1E-2

fig = go.Figure(layout=dict(width=1000, height=500, margin=go.layout.Margin(l=50, r=20, b=20, t=50, pad=0)),
                data=go.Parcoords(
    line=dict(color=df_plot['fitness'],
                colorscale=[[0, 'white'], [0.9,'lightgrey'], [0.91,'blue'], [0.96, 'orange'], [1, 'red']],
                showscale=True,
                cmin=min_all*1E2,
                cmax=max_all*1E2,
                colorbar={"title": dict(text="Accuracy (%)", side="right", font=dict(size=cbar_label_size)),
                          "tickvals": np.arange(10, 100+1, 10),
                          "tickfont": dict(size=cbar_ticks_size)}),
    dimensions=list([
        dict(tickvals=tickvals_a,
            label="a", values=df_plot['a']),
        dict(tickvals=tickvals_A1,
            label='A1', values=df_plot['A1']),
        dict(tickvals=tickvals_A2,
            label='A2', values=df_plot['A2']),
        dict(tickvals=tickvals_b,
            label='b', values=df_plot['b']),
        dict(tickvals=tickvals_G,
            label='G', values=df_plot['G']),
        dict(range=[0, 100],
             tickvals=np.arange(0, 100+1, 20),
             label='Accuracy (%)', values=df_plot['fitness'])
    ]),
    labelfont=dict(size=dimension_label_size),
    tickfont=dict(size=dimension_ticks_size)
)
)
image_filename = "bio_inspired_optimization_2_selection"

fig.write_image(f"./plots/{image_filename}.pdf")
fig.write_image(f"./plots/{image_filename}.png")
fig.show()
print("Nbr. trials highlighted: {}" .format(len(df_plot1)))
print("Nbr. generations: {}" .format(int(len(df_plot)/10)))


Nbr. trials highlighted: 34
Nbr. generations: 34


## Plot experiment 3
For this experiment the parameters "a", "A1", and "A2" have been optimized. The optimization used 10 individuals and 100 generations. 

In [46]:
file_storage_path = './results/experiment_3.pkl'

with open(file_storage_path, 'rb') as f:
    data = pickle.load(f)

# seperate best individual and data
best_individual = data[1::2]
data = data[::2]
# select only data from best individual
all_data = []
data_best_individual = []
for counter, individual in enumerate(data):
    all_data = np.append(all_data, individual)
    data_best_individual.append(individual[best_individual[counter]])
    
df_plot = pd.DataFrame.from_records(all_data)

In [47]:
df_plot.sort_values(by='fitness', ascending=False, inplace=True)

min_acc = np.round(np.nanmin(df_plot["fitness"]),-1)
min_bar = (int(str(min_acc)[0])-1)*10**(len(str(int(min_acc)))-1)

max_acc = np.round(np.nanmax(df_plot["fitness"]),-1)
max_bar = (int(str(max_acc)[0])+1)*10**(len(str(int(min_acc)))-1)

# fix number of ticks and set precision
tickvals_a = np.linspace(np.nanmin(df_plot['a'])*1.05, np.nanmax(df_plot['a'])*1.05, 10)
tickvals_a = tickvals_a.round(0)

tickvals_A1 = np.linspace(np.nanmin(df_plot['A1'])*1.05, np.nanmax(df_plot['A1'])*1.05, 10)
tickvals_A1 = tickvals_A1.round(2)

tickvals_A2 = np.linspace(np.nanmin(df_plot['A2'])*1.05, np.nanmax(df_plot['A2'])*1.05, 10)
tickvals_A2 = tickvals_A2.round(2)

In [48]:

fig = go.Figure(layout=dict(width=1000, height=500, margin=go.layout.Margin(l=50, r=20, b=20, t=50, pad=0)),
                data=go.Parcoords(
    line=dict(color=df_plot['fitness'],
                colorscale=px.colors.diverging.Portland,
                showscale=True,
                cmin=0,
                cmax=100,
                colorbar={"title": dict(text="Accuracy (%)", side="right", font=dict(size=cbar_label_size)),
                          "tickvals": np.arange(0, 100+1, 10),
                          "tickfont": dict(size=cbar_ticks_size)}),
    dimensions=list([
        dict(tickvals=tickvals_a,
            label="a", values=df_plot['a']),
        dict(tickvals=tickvals_A1,
            label='A1', values=df_plot['A1']),
        dict(tickvals=tickvals_A2,
            label='A2', values=df_plot['A2']),
        dict(range=[0, 100],
             tickvals=np.arange(0, 100+1, 20),
             label='Accuracy (%)', values=df_plot['fitness'])
    ]),
    labelfont=dict(size=dimension_label_size),
    tickfont=dict(size=dimension_ticks_size)
)
)
image_filename = "bio_inspired_optimization_3"

fig.write_image(f"./plots/{image_filename}.pdf")
fig.write_image(f"./plots/{image_filename}.png")
fig.show()
print("Nbr. trials: {}" .format(len(df_plot)))
print("Nbr. generations: {}" .format(int(len(df_plot)/10)))


Nbr. trials: 30
Nbr. generations: 3


### Highlight top 10%

In [49]:
df_plot1 = df_plot.iloc[:int(len(df_plot)*0.1), :]
df_plot2 = df_plot.iloc[int(len(df_plot)*0.1):, :]

In [50]:
min_top_ten = min(df_plot1["fitness"])*1E-2
max_top_ten = max(df_plot1["fitness"])*1E-2
min_all = min(df_plot["fitness"])*1E-2
max_all = max(df_plot["fitness"])*1E-2
# print(min_all, max_all, min_top_ten, (min_top_ten+max_top_ten)/2, max_top_ten)
# print((min_top_ten/max_all), (((min_top_ten+max_top_ten)/2)/max_all), (max_top_ten/max_all))
fig = go.Figure(layout=dict(width=1000, height=500, margin=go.layout.Margin(l=50, r=20, b=20, t=50, pad=0)),
                data=go.Parcoords(
    line=dict(color=df_plot['fitness'],
                colorscale=[[0, 'white'], [0.9,'lightgrey'], [0.91,'blue'], [0.96, 'orange'], [1, 'red']],
                showscale=True,
                cmin=min_all*1E2,
                cmax=max_all*1E2,
                colorbar={"title": dict(text="Accuracy (%)", side="right", font=dict(size=cbar_label_size)),
                          "tickvals": np.arange(10, 100+1, 10),
                          "tickfont": dict(size=cbar_ticks_size)}),
    dimensions=list([
        dict(tickvals=tickvals_a,
            label="a", values=df_plot['a']),
        dict(tickvals=tickvals_A1,
            label='A1', values=df_plot['A1']),
        dict(tickvals=tickvals_A2,
            label='A2', values=df_plot['A2']),
        dict(range=[0, 100],
             tickvals=np.arange(0, 100+1, 20),
             label='Accuracy (%)', values=df_plot['fitness'])
    ]),
    labelfont=dict(size=dimension_label_size),
    tickfont=dict(size=dimension_ticks_size)
)
)
image_filename = "bio_inspired_optimization_3_selection"

fig.write_image(f"./plots/{image_filename}.pdf")
fig.write_image(f"./plots/{image_filename}.png")
fig.show()
print("Nbr. trials highlighted: {}" .format(len(df_plot1)))
print("Nbr. generations: {}" .format(int(len(df_plot)/10)))


Nbr. trials highlighted: 3
Nbr. generations: 3


## Plot experiment 4
For this experiment the parameters "a", "A1", and "A2" have been optimized. The optimization used 25 individuals and 100 generations. 

In [51]:
file_storage_path = './results/experiment_4.pkl'

with open(file_storage_path, 'rb') as f:
    data = pickle.load(f)

# seperate best individual and data
best_individual = data[1::2]
data = data[::2]
# select only data from best individual
all_data = []
data_best_individual = []
for counter, individual in enumerate(data):
    all_data = np.append(all_data, individual)
    data_best_individual.append(individual[best_individual[counter]])
    
df_plot = pd.DataFrame.from_records(all_data)

In [52]:
df_plot.sort_values(by='fitness', ascending=False, inplace=True)

min_acc = np.round(np.nanmin(df_plot["fitness"]),-1)
min_bar = (int(str(min_acc)[0])-1)*10**(len(str(int(min_acc)))-1)

max_acc = np.round(np.nanmax(df_plot["fitness"]),-1)
max_bar = (int(str(max_acc)[0])+1)*10**(len(str(int(min_acc)))-1)

# fix number of ticks and set precision
tickvals_a = np.linspace(np.nanmin(df_plot['a'])*1.05, np.nanmax(df_plot['a'])*1.05, 10)
tickvals_a = tickvals_a.round(0)

tickvals_A1 = np.linspace(np.nanmin(df_plot['A1'])*1.05, np.nanmax(df_plot['A1'])*1.05, 10)
tickvals_A1 = tickvals_A1.round(2)

tickvals_A2 = np.linspace(np.nanmin(df_plot['A2'])*1.05, np.nanmax(df_plot['A2'])*1.05, 10)
tickvals_A2 = tickvals_A2.round(2)

In [53]:

fig = go.Figure(layout=dict(width=1000, height=500, margin=go.layout.Margin(l=50, r=20, b=20, t=50, pad=0)),
                data=go.Parcoords(
    line=dict(color=df_plot['fitness'],
                colorscale=px.colors.diverging.Portland,
                showscale=True,
                cmin=0,
                cmax=100,
                colorbar={"title": dict(text="Accuracy (%)", side="right", font=dict(size=cbar_label_size)),
                          "tickvals": np.arange(0, 100+1, 10),
                          "tickfont": dict(size=cbar_ticks_size)}),
    dimensions=list([
        dict(tickvals=tickvals_a,
            label="a", values=df_plot['a']),
        dict(tickvals=tickvals_A1,
            label='A1', values=df_plot['A1']),
        dict(tickvals=tickvals_A2,
            label='A2', values=df_plot['A2']),
        dict(range=[0, 100],
             tickvals=np.arange(0, 100+1, 20),
             label='Accuracy (%)', values=df_plot['fitness'])
    ]),
    labelfont=dict(size=dimension_label_size),
    tickfont=dict(size=dimension_ticks_size)
)
)
image_filename = "bio_inspired_optimization_4"

fig.write_image(f"./plots/{image_filename}.pdf")
fig.write_image(f"./plots/{image_filename}.png")
fig.show()
print("Nbr. trials: {}" .format(len(df_plot)))
print("Nbr. generations: {}" .format(int(len(df_plot)/10)))


Nbr. trials: 150
Nbr. generations: 15


### Highlight top 10%

In [54]:
df_plot1 = df_plot.iloc[:int(len(df_plot)*0.1), :]
df_plot2 = df_plot.iloc[int(len(df_plot)*0.1):, :]

In [55]:
min_top_ten = min(df_plot1["fitness"])*1E-2
max_top_ten = max(df_plot1["fitness"])*1E-2
min_all = min(df_plot["fitness"])*1E-2
max_all = max(df_plot["fitness"])*1E-2
# print(min_all, max_all, min_top_ten, (min_top_ten+max_top_ten)/2, max_top_ten)
# print((min_top_ten/max_all), (((min_top_ten+max_top_ten)/2)/max_all), (max_top_ten/max_all))
fig = go.Figure(layout=dict(width=1000, height=500, margin=go.layout.Margin(l=50, r=20, b=20, t=50, pad=0)),
                data=go.Parcoords(
    line=dict(color=df_plot['fitness'],
                colorscale=[[0, 'white'], [0.9,'lightgrey'], [0.91,'blue'], [0.96, 'orange'], [1, 'red']],
                showscale=True,
                cmin=min_all*1E2,
                cmax=max_all*1E2,
                colorbar={"title": dict(text="Accuracy (%)", side="right", font=dict(size=cbar_label_size)),
                          "tickvals": np.arange(10, 100+1, 10),
                          "tickfont": dict(size=cbar_ticks_size)}),
    dimensions=list([
        dict(tickvals=tickvals_a,
            label="a", values=df_plot['a']),
        dict(tickvals=tickvals_A1,
            label='A1', values=df_plot['A1']),
        dict(tickvals=tickvals_A2,
            label='A2', values=df_plot['A2']),
        dict(range=[0, 100],
             tickvals=np.arange(0, 100+1, 20),
             label='Accuracy (%)', values=df_plot['fitness'])
    ]),
    labelfont=dict(size=dimension_label_size),
    tickfont=dict(size=dimension_ticks_size)
)
)
image_filename = "bio_inspired_optimization_4_selection"

fig.write_image(f"./plots/{image_filename}.pdf")
fig.write_image(f"./plots/{image_filename}.png")
fig.show()
print("Nbr. trials highlighted: {}" .format(len(df_plot1)))
print("Nbr. generations: {}" .format(int(len(df_plot)/10)))


Nbr. trials highlighted: 15
Nbr. generations: 15


## Plot experiment 5
For this experiment the parameters "a", "A1", and "A2" have been optimized. The optimization used 25 individuals and 100 generations. 
Searchspace is limited following inverse sigmoid function.

In [56]:
file_storage_path = './results/experiment_5.pkl'

with open(file_storage_path, 'rb') as f:
    data = pickle.load(f)

# seperate best individual and data
best_individual = data[1::2]
data = data[::2]
# select only data from best individual
all_data = []
data_best_individual = []
for counter, individual in enumerate(data):
    all_data = np.append(all_data, individual)
    data_best_individual.append(individual[best_individual[counter]])
    
df_plot = pd.DataFrame.from_records(all_data)

In [57]:
df_plot.sort_values(by='fitness', ascending=False, inplace=True)

min_acc = np.round(np.nanmin(df_plot["fitness"]),-1)
min_bar = (int(str(min_acc)[0])-1)*10**(len(str(int(min_acc)))-1)

max_acc = np.round(np.nanmax(df_plot["fitness"]),-1)
max_bar = (int(str(max_acc)[0])+1)*10**(len(str(int(min_acc)))-1)

# fix number of ticks and set precision
tickvals_a = np.linspace(np.nanmin(df_plot['a'])*1.05, np.nanmax(df_plot['a'])*1.05, 10)
tickvals_a = tickvals_a.round(0)

tickvals_A1 = np.linspace(np.nanmin(df_plot['A1'])*1.05, np.nanmax(df_plot['A1'])*1.05, 10)
tickvals_A1 = tickvals_A1.round(2)

tickvals_A2 = np.linspace(np.nanmin(df_plot['A2'])*1.05, np.nanmax(df_plot['A2'])*1.05, 10)
tickvals_A2 = tickvals_A2.round(2)

In [58]:

fig = go.Figure(layout=dict(width=1000, height=500, margin=go.layout.Margin(l=50, r=20, b=20, t=50, pad=0)),
                data=go.Parcoords(
    line=dict(color=df_plot['fitness'],
                colorscale=px.colors.diverging.Portland,
                showscale=True,
                cmin=0,
                cmax=100,
                colorbar={"title": dict(text="Accuracy (%)", side="right", font=dict(size=cbar_label_size)),
                          "tickvals": np.arange(0, 100+1, 10),
                          "tickfont": dict(size=cbar_ticks_size)}),
    dimensions=list([
        dict(tickvals=tickvals_a,
            label="a", values=df_plot['a']),
        dict(tickvals=tickvals_A1,
            label='A1', values=df_plot['A1']),
        dict(tickvals=tickvals_A2,
            label='A2', values=df_plot['A2']),
        dict(range=[0, 100],
             tickvals=np.arange(0, 100+1, 20),
             label='Accuracy (%)', values=df_plot['fitness'])
    ]),
    labelfont=dict(size=dimension_label_size),
    tickfont=dict(size=dimension_ticks_size)
)
)
image_filename = "bio_inspired_optimization_5"

fig.write_image(f"./plots/{image_filename}.pdf")
fig.write_image(f"./plots/{image_filename}.png")
fig.show()
print("Nbr. trials: {}" .format(len(df_plot)))
print("Nbr. generations: {}" .format(int(len(df_plot)/10)))


Nbr. trials: 400
Nbr. generations: 40


### Highlight top 10%

In [59]:
df_plot1 = df_plot.iloc[:int(len(df_plot)*0.1), :]
df_plot2 = df_plot.iloc[int(len(df_plot)*0.1):, :]

In [60]:
min_top_ten = min(df_plot1["fitness"])*1E-2
max_top_ten = max(df_plot1["fitness"])*1E-2
min_all = min(df_plot["fitness"])*1E-2
max_all = max(df_plot["fitness"])*1E-2
# print(min_all, max_all, min_top_ten, (min_top_ten+max_top_ten)/2, max_top_ten)
# print((min_top_ten/max_all), (((min_top_ten+max_top_ten)/2)/max_all), (max_top_ten/max_all))
fig = go.Figure(layout=dict(width=1000, height=500, margin=go.layout.Margin(l=50, r=20, b=20, t=50, pad=0)),
                data=go.Parcoords(
    line=dict(color=df_plot['fitness'],
                colorscale=[[0, 'white'], [0.9,'lightgrey'], [0.91,'blue'], [0.96, 'orange'], [1, 'red']],
                showscale=True,
                cmin=min_all*1E2,
                cmax=max_all*1E2,
                colorbar={"title": dict(text="Accuracy (%)", side="right", font=dict(size=cbar_label_size)),
                          "tickvals": np.arange(10, 100+1, 10),
                          "tickfont": dict(size=cbar_ticks_size)}),
    dimensions=list([
        dict(tickvals=tickvals_a,
            label="a", values=df_plot['a']),
        dict(tickvals=tickvals_A1,
            label='A1', values=df_plot['A1']),
        dict(tickvals=tickvals_A2,
            label='A2', values=df_plot['A2']),
        dict(range=[0, 100],
             tickvals=np.arange(0, 100+1, 20),
             label='Accuracy (%)', values=df_plot['fitness'])
    ]),
    labelfont=dict(size=dimension_label_size),
    tickfont=dict(size=dimension_ticks_size)
)
)
image_filename = "bio_inspired_optimization_5_selection"

fig.write_image(f"./plots/{image_filename}.pdf")
fig.write_image(f"./plots/{image_filename}.png")
fig.show()
print("Nbr. trials highlighted: {}" .format(len(df_plot1)))
print("Nbr. generations: {}" .format(int(len(df_plot)/10)))


Nbr. trials highlighted: 40
Nbr. generations: 40


## Plot experiment 6
For this experiment the parameters "a", "A1", and "A2" have been optimized. The optimization used 100 individuals and 100 generations. 
Searchspace is limited following inverse sigmoid function.

In [61]:
file_storage_path = './results/experiment_6.pkl'

with open(file_storage_path, 'rb') as f:
    data = pickle.load(f)

# seperate best individual and data
best_individual = data[1::2]
data = data[::2]
# select only data from best individual
all_data = []
data_best_individual = []
for counter, individual in enumerate(data):
    all_data = np.append(all_data, individual)
    data_best_individual.append(individual[best_individual[counter]])
    
df_plot = pd.DataFrame.from_records(all_data)

In [62]:
df_plot.sort_values(by='fitness', ascending=False, inplace=True)

min_acc = np.round(np.nanmin(df_plot["fitness"]),-1)
min_bar = (int(str(min_acc)[0])-1)*10**(len(str(int(min_acc)))-1)

max_acc = np.round(np.nanmax(df_plot["fitness"]),-1)
max_bar = (int(str(max_acc)[0])+1)*10**(len(str(int(min_acc)))-1)

# fix number of ticks and set precision
tickvals_a = np.linspace(np.nanmin(df_plot['a'])*1.05, np.nanmax(df_plot['a'])*1.05, 10)
tickvals_a = tickvals_a.round(0)

tickvals_A1 = np.linspace(np.nanmin(df_plot['A1'])*1.05, np.nanmax(df_plot['A1'])*1.05, 10)
tickvals_A1 = tickvals_A1.round(2)

tickvals_A2 = np.linspace(np.nanmin(df_plot['A2'])*1.05, np.nanmax(df_plot['A2'])*1.05, 10)
tickvals_A2 = tickvals_A2.round(2)

In [63]:

fig = go.Figure(layout=dict(width=1000, height=500, margin=go.layout.Margin(l=50, r=20, b=20, t=50, pad=0)),
                data=go.Parcoords(
    line=dict(color=df_plot['fitness'],
                colorscale=px.colors.diverging.Portland,
                showscale=True,
                cmin=0,
                cmax=100,
                colorbar={"title": dict(text="Accuracy (%)", side="right", font=dict(size=cbar_label_size)),
                          "tickvals": np.arange(0, 100+1, 10),
                          "tickfont": dict(size=cbar_ticks_size)}),
    dimensions=list([
        dict(tickvals=tickvals_a,
            label="a", values=df_plot['a']),
        dict(tickvals=tickvals_A1,
            label='A1', values=df_plot['A1']),
        dict(tickvals=tickvals_A2,
            label='A2', values=df_plot['A2']),
        dict(range=[0, 100],
             tickvals=np.arange(0, 100+1, 20),
             label='Accuracy (%)', values=df_plot['fitness'])
    ]),
    labelfont=dict(size=dimension_label_size),
    tickfont=dict(size=dimension_ticks_size)
)
)
image_filename = "bio_inspired_optimization_6"

fig.write_image(f"./plots/{image_filename}.pdf")
fig.write_image(f"./plots/{image_filename}.png")
fig.show()
print("Nbr. trials: {}" .format(len(df_plot)))
print("Nbr. generations: {}" .format(int(len(df_plot)/10)))


Nbr. trials: 200
Nbr. generations: 20


### Highlight top 10%

In [64]:
df_plot1 = df_plot.iloc[:int(len(df_plot)*0.1), :]
df_plot2 = df_plot.iloc[int(len(df_plot)*0.1):, :]

In [65]:
min_top_ten = min(df_plot1["fitness"])*1E-2
max_top_ten = max(df_plot1["fitness"])*1E-2
min_all = min(df_plot["fitness"])*1E-2
max_all = max(df_plot["fitness"])*1E-2
# print(min_all, max_all, min_top_ten, (min_top_ten+max_top_ten)/2, max_top_ten)
# print((min_top_ten/max_all), (((min_top_ten+max_top_ten)/2)/max_all), (max_top_ten/max_all))
fig = go.Figure(layout=dict(width=1000, height=500, margin=go.layout.Margin(l=50, r=20, b=20, t=50, pad=0)),
                data=go.Parcoords(
    line=dict(color=df_plot['fitness'],
                colorscale=[[0, 'white'], [0.9,'lightgrey'], [0.91,'blue'], [0.96, 'orange'], [1, 'red']],
                showscale=True,
                cmin=min_all*1E2,
                cmax=max_all*1E2,
                colorbar={"title": dict(text="Accuracy (%)", side="right", font=dict(size=cbar_label_size)),
                          "tickvals": np.arange(10, 100+1, 10),
                          "tickfont": dict(size=cbar_ticks_size)}),
    dimensions=list([
        dict(tickvals=tickvals_a,
            label="a", values=df_plot['a']),
        dict(tickvals=tickvals_A1,
            label='A1', values=df_plot['A1']),
        dict(tickvals=tickvals_A2,
            label='A2', values=df_plot['A2']),
        dict(range=[0, 100],
             tickvals=np.arange(0, 100+1, 20),
             label='Accuracy (%)', values=df_plot['fitness'])
    ]),
    labelfont=dict(size=dimension_label_size),
    tickfont=dict(size=dimension_ticks_size)
)
)
image_filename = "bio_inspired_optimization_6_selection"

fig.write_image(f"./plots/{image_filename}.pdf")
fig.write_image(f"./plots/{image_filename}.png")
fig.show()
print("Nbr. trials highlighted: {}" .format(len(df_plot1)))
print("Nbr. generations: {}" .format(int(len(df_plot)/10)))


Nbr. trials highlighted: 20
Nbr. generations: 20


## Plot experiment 7
For this experiment the parameters "a", "A1", and "A2" have been optimized. The optimization used 100 individuals and 100 generations. 
Searchspace is limited following inverse sigmoid function.

In [ ]:
file_storage_path = './results/experiment_7.pkl'

with open(file_storage_path, 'rb') as f:
    data = pickle.load(f)

# seperate best individual and data
best_individual = data[1::2]
data = data[::2]
# select only data from best individual
all_data = []
data_best_individual = []
for counter, individual in enumerate(data):
    all_data = np.append(all_data, individual)
    data_best_individual.append(individual[best_individual[counter]])
    
df_plot = pd.DataFrame.from_records(all_data)

In [ ]:
df_plot.sort_values(by='fitness', ascending=False, inplace=True)

min_acc = np.round(np.nanmin(df_plot["fitness"]),-1)
min_bar = (int(str(min_acc)[0])-1)*10**(len(str(int(min_acc)))-1)

max_acc = np.round(np.nanmax(df_plot["fitness"]),-1)
max_bar = (int(str(max_acc)[0])+1)*10**(len(str(int(min_acc)))-1)

# fix number of ticks and set precision
tickvals_a = np.linspace(np.nanmin(df_plot['a'])*1.05, np.nanmax(df_plot['a'])*1.05, 10)
tickvals_a = tickvals_a.round(0)

tickvals_A1 = np.linspace(np.nanmin(df_plot['A1'])*1.05, np.nanmax(df_plot['A1'])*1.05, 10)
tickvals_A1 = tickvals_A1.round(2)

tickvals_A2 = np.linspace(np.nanmin(df_plot['A2'])*1.05, np.nanmax(df_plot['A2'])*1.05, 10)
tickvals_A2 = tickvals_A2.round(2)

In [ ]:

fig = go.Figure(layout=dict(width=1000, height=500, margin=go.layout.Margin(l=50, r=20, b=20, t=50, pad=0)),
                data=go.Parcoords(
    line=dict(color=df_plot['fitness'],
                colorscale=px.colors.diverging.Portland,
                showscale=True,
                cmin=0,
                cmax=100,
                colorbar={"title": dict(text="Accuracy (%)", side="right", font=dict(size=cbar_label_size)),
                          "tickvals": np.arange(0, 100+1, 10),
                          "tickfont": dict(size=cbar_ticks_size)}),
    dimensions=list([
        dict(tickvals=tickvals_a,
            label="a", values=df_plot['a']),
        dict(tickvals=tickvals_A1,
            label='A1', values=df_plot['A1']),
        dict(tickvals=tickvals_A2,
            label='A2', values=df_plot['A2']),
        dict(range=[0, 100],
             tickvals=np.arange(0, 100+1, 20),
             label='Accuracy (%)', values=df_plot['fitness'])
    ]),
    labelfont=dict(size=dimension_label_size),
    tickfont=dict(size=dimension_ticks_size)
)
)
image_filename = "bio_inspired_optimization_7"

fig.write_image(f"./plots/{image_filename}.pdf")
fig.write_image(f"./plots/{image_filename}.png")
fig.show()
print("Nbr. trials: {}" .format(len(df_plot)))
print("Nbr. generations: {}" .format(int(len(df_plot)/10)))


### Highlight top 10%

In [ ]:
df_plot1 = df_plot.iloc[:int(len(df_plot)*0.1), :]
df_plot2 = df_plot.iloc[int(len(df_plot)*0.1):, :]

In [ ]:
min_top_ten = min(df_plot1["fitness"])*1E-2
max_top_ten = max(df_plot1["fitness"])*1E-2
min_all = min(df_plot["fitness"])*1E-2
max_all = max(df_plot["fitness"])*1E-2
# print(min_all, max_all, min_top_ten, (min_top_ten+max_top_ten)/2, max_top_ten)
# print((min_top_ten/max_all), (((min_top_ten+max_top_ten)/2)/max_all), (max_top_ten/max_all))
fig = go.Figure(layout=dict(width=1000, height=500, margin=go.layout.Margin(l=50, r=20, b=20, t=50, pad=0)),
                data=go.Parcoords(
    line=dict(color=df_plot['fitness'],
                colorscale=[[0, 'white'], [0.9,'lightgrey'], [0.91,'blue'], [0.96, 'orange'], [1, 'red']],
                showscale=True,
                cmin=min_all*1E2,
                cmax=max_all*1E2,
                colorbar={"title": dict(text="Accuracy (%)", side="right", font=dict(size=cbar_label_size)),
                          "tickvals": np.arange(10, 100+1, 10),
                          "tickfont": dict(size=cbar_ticks_size)}),
    dimensions=list([
        dict(tickvals=tickvals_a,
            label="a", values=df_plot['a']),
        dict(tickvals=tickvals_A1,
            label='A1', values=df_plot['A1']),
        dict(tickvals=tickvals_A2,
            label='A2', values=df_plot['A2']),
        dict(range=[0, 100],
             tickvals=np.arange(0, 100+1, 20),
             label='Accuracy (%)', values=df_plot['fitness'])
    ]),
    labelfont=dict(size=dimension_label_size),
    tickfont=dict(size=dimension_ticks_size)
)
)
image_filename = "bio_inspired_optimization_7_selection"

fig.write_image(f"./plots/{image_filename}.pdf")
fig.write_image(f"./plots/{image_filename}.png")
fig.show()
print("Nbr. trials highlighted: {}" .format(len(df_plot1)))
print("Nbr. generations: {}" .format(int(len(df_plot)/10)))
